In [1]:
import random
import dlib, cv2, os
import pandas as pd
import numpy as np
from helper import resize

for i in range(7):
  print(i,'번째 전처리..')
  dirname = 'CAT_0'+str(i)
  base_path = 'dataset/cats/%s' % dirname
  file_list = sorted(os.listdir(base_path))
  random.shuffle(file_list)

  dataset = {
    'imgs': [],
    'lmks': [],
    'bbs': []
  }

  for f in file_list:
    if '.cat' not in f:
      continue

    # read landmarks
    pd_frame = pd.read_csv(os.path.join(base_path, f), sep=' ', header=None)
    landmarks = (pd_frame.to_numpy()[0][1:-1]).reshape((-1, 2))

    # load image
    img_filename, ext = os.path.splitext(f)

    img = cv2.imread(os.path.join(base_path, img_filename))

    # resize image and relocate landmarks
    img, ratio, top, left = resize.resize_img(img)
    # 변한 랜드마크를 재계산
    landmarks = ((landmarks * ratio) + np.array([left, top])).astype(np.int32)
    # 얼굴의 영역 지정
    bb = np.array([np.min(landmarks, axis=0), np.max(landmarks, axis=0)])

    dataset['imgs'].append(img)
    dataset['lmks'].append(landmarks.flatten())
    dataset['bbs'].append(bb.flatten())

    # for l in landmarks:
    #   cv2.circle(img, center=tuple(l), radius=1, color=(255, 255, 255), thickness=2)

    # cv2.imshow('img', img)
    # if cv2.waitKey(0) == ord('q'):
    #   break

  np.save('dataset/%s.npy' % dirname, np.array(dataset))

0 번째 전처리..
1 번째 전처리..
2 번째 전처리..
3 번째 전처리..
4 번째 전처리..
5 번째 전처리..
6 번째 전처리..


In [ ]:
import keras, datetime
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from keras.applications import mobilenet_v2
import numpy as np
import tensorflow as tf

print(tf.__version__)
print(keras.__version__)

img_size = 224

mode = 'bbs' # [bbs, lmks]
if mode is 'bbs':
  output_size = 4
elif mode is 'lmks':
  output_size = 18

print('output size : ',output_size)

start_time = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

print('dataloads start!')

data_00 = np.load('dataset/CAT_00.npy')
data_01 = np.load('dataset/CAT_01.npy')
data_02 = np.load('dataset/CAT_02.npy')
data_03 = np.load('dataset/CAT_03.npy')
data_04 = np.load('dataset/CAT_04.npy')
data_05 = np.load('dataset/CAT_05.npy')
data_06 = np.load('dataset/CAT_06.npy')

print('dataloads finish!')
print('data preprocessing start!')

# 0~5번째의 사진들을 train set으로 사용
x_train = np.concatenate((data_00.item().get('imgs'), data_01.item().get('imgs'), data_02.item().get('imgs'), data_03.item().get('imgs'), data_04.item().get('imgs'), data_05.item().get('imgs')), axis=0)
y_train = np.concatenate((data_00.item().get(mode), data_01.item().get(mode), data_02.item().get(mode), data_03.item().get(mode), data_04.item().get(mode), data_05.item().get(mode)), axis=0)

# 6번째의 사진들을 test set으로 사용
x_test = np.array(data_06.item().get('imgs'))
y_test = np.array(data_06.item().get(mode))

# 이미지를 0~1로 바꿔줌 (normalization), 학습모델의 크기에 맞게 reshape해줌
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (-1, img_size, img_size, 3))
x_test = np.reshape(x_test, (-1, img_size, img_size, 3))

y_train = np.reshape(y_train, (-1, output_size))
y_test = np.reshape(y_test, (-1, output_size))

inputs = Input(shape=(img_size, img_size, 3))

print('data preprocessing finish!')
print('model build start!')

mobilenetv2_model = mobilenet_v2.MobileNetV2(input_shape=(img_size, img_size, 3), alpha=1.0,  include_top=False, weights='imagenet', input_tensor=inputs, pooling='max')

net = Dense(128, activation='relu')(mobilenetv2_model.layers[-1].output)
net = Dense(64, activation='relu')(net)
net = Dense(output_size, activation='linear')(net)

model = Model(inputs=inputs, outputs=net)

model.summary()

print('model build finish!')
print('model training start!')

# training
model.compile(optimizer=keras.optimizers.Adam(), loss='mse')

# 학습시간이 너무길어 epochs=50 -> 5로 변경
model.fit(x_train, y_train, epochs=5, batch_size=32, shuffle=True,
  validation_data=(x_test, y_test), verbose=1,
  callbacks=[
    TensorBoard(log_dir='logs/%s' % (start_time)),
    ModelCheckpoint('./models/%s.h5' % (start_time), monitor='val_loss', verbose=1, save_best_only=True, mode='auto'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, mode='auto')
  ]
)

print('model training finish!')

2.11.0
2.11.0
output size :  4
dataloads start!
dataloads finish!
data preprocessing start!
data preprocessing finish!
model build start!
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  12

                                                                                                  
 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
          

 block_6_project_BN (BatchNorma  (None, 14, 14, 64)  256         ['block_6_project[0][0]']        
 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_d

 rmalization)                                                                                     
                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 576)  55296       ['block_10_project_BN[0][0]']    
          

                                                                                                  
 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 160)    153600      ['block_14_depthwise_relu[0][0]']
          